In [1]:
!pip install pyDOE
!pip install cma


  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=a1f80f51dfc0f94166057f0eb7be6a3d41b901533c71baf70fa54459300a0d14
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.8MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import PolynomialFeatures
import bbobbenchmarks as bn
import sklearn

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F20 '''
def F20(X):
    f = bn.F20()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data

''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))
      

''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 13.06 ,random_state=0 , l1_ratio=0.997, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 69.69 , C = 1000.000000 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_202000Samples.csv"
train_y = pd.read_csv(path).iloc[:,-1]
test_y = pd.read_csv('test_20_400Samples.csv').iloc[:,-1]
path = "train_latent_100D.csv"
train = pd.read_csv(path, index_col = 0)
test = pd.read_csv('test_latent_100D.csv', index_col = 0)
train ['Y'] = train_y
test ['Y'] = test_y
del train_y
del test_y
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_Z'+str(i))

opt = cma.CMAOptions()
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
    print ('Run : '+ str(i))
    min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 0.2689433305566933 , options=opt) [0]
    X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

Run : 0
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=957169, Mon Apr 13 14:48:48 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 9.959850111045945e+05 1.0e+00 2.53e-01  3e-01  3e-01 0:00.0
    2     30 9.945537363540014e+05 1.0e+00 2.46e-01  2e-01  2e-01 0:00.0
    3     45 9.892164900651794e+05 1.1e+00 2.40e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [996229.57105159] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [996277.56787071] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -7.170094795807762e+05 1.4e+00 2.51e-02  2e-02  3e-02 0:01.9
  200   3000 -7.560750077193020e+05 1.5e+00 1.28e-03  1e-03  1e-03 0:03.9
  300   4500 -7.561668478535715e+05 1.5e+00 7.03e-05  6e-05  7e-05 0:05.8
  400   6000 -7.561671147927096e+05 1.6e+00 4.84e-06  4e-06  4e-06 0:07.7
  500   7500 -7.561671158728091e+05 1.6e+00 3.21e-07  3e-07  3e-07 0:09.7
  600   9000 -7.561671158785288e+05 1.7e+00 2.11e-08  2e-08  2e-08 0:11.6
  700  10500 -7.561671158785444e+05 1.8e+00 2.49e-09  2e-09  2e-09 0:13.6
  800  12000 -7.561671158785442e+05 1.9e+00 1.21e-09  9e-10  1e-09 0:15.5
  900  13500 -7.561671158785442e+05 2.3e+00 1.50e-09  1e-09  1e-09 0:17.5
 1000  15000 -7.561671158785442e+05 2.4e+00 1.06e-09  7e-10  9e-10 0:19.4
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -7.561671158785446e+05 2.4e+00 6.67e-10  4e-10  6e-10 0:21.3
 1200  18000 -7.561671158785446e+05 2.8e+00 1.02e-09  7e-10  9e-10 0:23.3
 1300  19500 -7.561671158785446e+05 2.9e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [980953.14023324] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=980953.140233, sigma=2.55e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=980953.140233, sigma=2.55e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)


  100   1500 -3.641647045923024e+11 1.4e+00 1.81e+02  2e+02  2e+02 0:01.8
  106   1590 -9.195771770294427e+11 1.4e+00 2.66e+02  2e+02  3e+02 0:02.0
termination on tolfacupx=1000.0 (Mon Apr 13 14:49:59 2020)
final/bestever f-value = -8.421741e+11 -9.195772e+11
incumbent solution: [ 186.51479927  409.05584627 -114.13310317 -537.20619449  518.97280732
 -544.65543992  109.33105505 -178.23697635 ...]
std deviations: [259.72707301 260.01736939 258.69765098 256.96295816 257.51351557
 257.775525   260.60364856 252.95703522 ...]
Run : 1
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=911237, Mon Apr 13 14:49:59 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 9.840080082870515e+05 1.0e+00 2.53e-01  3e-01  3e-01 0:00.0
    2     30 9.844533248106601e+05 1.0e+00 2.42e-01  2e-01  2e-01 0:00.0
    3     45 9.657282293660498e+05 1.0e+00 2.37e-01  2e-01  2e-01 0:00.1


...

   96   1440 -9.025328442280620e+11 1.4e+00 2.85e+02  3e+02  3e+02 0:01.8
termination on tolfacupx=1000.0 (Mon Apr 13 15:22:14 2020)
final/bestever f-value = -8.716111e+11 -9.025328e+11
incumbent solution: [  20.83702337  784.81153794  207.06262899 -344.75759326  858.34316551
 -446.80322472  376.29092393 -524.16418509 ...]
std deviations: [271.94470825 278.16700477 281.34637697 275.68158429 277.6946687
 276.46844573 281.21754566 274.26918183 ...]
Run : 29
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=861515, Mon Apr 13 15:22:14 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 9.948364836233244e+05 1.0e+00 2.55e-01  3e-01  3e-01 0:00.0
    2     30 9.932499215916328e+05 1.0e+00 2.46e-01  2e-01  2e-01 0:00.0
    3     45 9.888772231561703e+05 1.1e+00 2.39e-01  2e-01  2e-01 0:00.1


/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [996137.96165712] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [995283.86511938] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')


  100   1500 -7.281132793725691e+05 1.4e+00 2.18e-02  2e-02  2e-02 0:02.0
  200   3000 -7.560797431941581e+05 1.4e+00 1.19e-03  1e-03  1e-03 0:04.0
  300   4500 -7.561667638525401e+05 1.4e+00 7.91e-05  7e-05  8e-05 0:06.0
  400   6000 -7.561671143714424e+05 1.5e+00 5.75e-06  5e-06  5e-06 0:07.9
  500   7500 -7.561671158748701e+05 1.6e+00 3.10e-07  2e-07  3e-07 0:09.9
  600   9000 -7.561671158785269e+05 1.6e+00 2.10e-08  2e-08  2e-08 0:11.9
  700  10500 -7.561671158785444e+05 1.7e+00 2.26e-09  2e-09  2e-09 0:13.9
  800  12000 -7.561671158785449e+05 2.0e+00 1.71e-09  1e-09  1e-09 0:15.9
  900  13500 -7.561671158785449e+05 2.2e+00 2.15e-09  2e-09  2e-09 0:17.9
 1000  15000 -7.561671158785446e+05 2.3e+00 1.55e-09  1e-09  1e-09 0:19.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -7.561671158785446e+05 2.5e+00 1.18e-09  8e-10  1e-09 0:21.9
 1200  18000 -7.561671158785446e+05 2.8e+00 1.21e-09  8e-10  1e-09 0:23.9
 1300  19500 -7.561671158785449e+05 2.8e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [980953.14023324] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=980953.140233, sigma=2.55e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=980953.140233, sigma=2.55e-01).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)


  100   1500 -2.979996388814281e+11 1.5e+00 1.74e+02  2e+02  2e+02 0:01.8
  106   1590 -7.497849558391660e+11 1.5e+00 2.68e+02  3e+02  3e+02 0:01.9
termination on tolfacupx=1000.0 (Mon Apr 13 15:23:21 2020)
final/bestever f-value = -7.053072e+11 -7.497850e+11
incumbent solution: [-197.59860126  755.25511854 -188.97827578 -551.27191902 1122.24284075
  -48.04116259  621.74620399  151.06485333 ...]
std deviations: [257.22681249 263.83928987 257.68586153 271.06170932 268.56945994
 263.57977834 261.41874839 255.0989687  ...]


In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('F20_100_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F20(X_Values.iloc[i,:50])
    SVM_Fun [i] = F20(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F20(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([40430.48284913]), count=array([1])) 8.093217318860087e-05
SVM
ModeResult(mode=array([50715.36127618]), count=array([1])) 12064.49920076678
ELN
ModeResult(mode=array([3.24695715e+10]), count=array([1])) 12212966845.578646
